In [1]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense
from keras import optimizers
import numpy as np
import time

# models
from keras.applications.inception_v3 import InceptionV3

Using TensorFlow backend.


### Variables

In [2]:
classes = ['normal','tumor'] 
nb_classes = len(classes)

img_width, img_height = 200, 200
nb_epoch = 10

In [3]:
train_data_dir      = 'dataset/train'
validation_data_dir = 'dataset/validation'

nb_train_samples      = 2000
nb_validation_samples = 600

result_dir = 'dataset/results'
if not os.path.exists(result_dir):
    os.mkdir(result_dir)

### Model

In [4]:
def model_maker():

    input_tensor = Input(shape=(img_width, img_height, 3))
    inception = InceptionV3(include_top=False, weights='imagenet', input_tensor=input_tensor)

    x = inception.output
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(nb_classes, activation='softmax')(x)
    
    return Model(inputs=inception.input, outputs=predictions)


In [5]:
def image_generator():
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        zoom_range=0.0,
        rotation_range=0,
        width_shift_range=0.0,
        height_shift_range=0.0,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='reflect')

    validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)

    validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        color_mode='rgb',
        classes=classes,
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True)

    return (train_generator, validation_generator)

### Train

In [6]:
from keras.callbacks import ModelCheckpoint
import random

# Run only 5 loops to avoid memory leak
for i in range(1, 5):

    model = model_maker()

    checkpointer = ModelCheckpoint('ex1-result/finetuning' + str(i) + '.h5', monitor='val_acc', verbose=0, save_best_only=True)    

    batch_size = np.random.choice([16, 32, 64, 128])

    model.compile(loss='categorical_crossentropy',
                      optimizer=optimizers.Adam(),
                      metrics=['accuracy'])

    train_generator, validation_generator = image_generator()

    start = time.time()

    hist = model.fit_generator(
        train_generator,
        steps_per_epoch = nb_train_samples/batch_size,
        validation_steps = nb_validation_samples/batch_size,
        nb_epoch = nb_epoch,
        validation_data = validation_generator,
        callbacks=[checkpointer],
        verbose=0)

    model.save("ex1-result/model" + str(i) + ".h5")
    
    print(i, batch_size, checkpointer.best)
    f = open('ex1-result/result.txt','a')
    f.write(str(i) +  ',' + str(batch_size) + ',' + str(checkpointer.best) + '\n')
    f.close()
    model = None


Found 2000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:28: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:28: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_steps=18.75, callbacks=[<keras.ca..., steps_per_epoch=62.5, validation_data=<keras.pre..., verbose=0, epochs=10)`


1 32 0.926666667461
Found 2000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
2 32 0.961666665872
Found 2000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
3 32 0.943333333333
Found 2000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
4 32 0.946666667461


In [12]:
# 試行の中の最高のモデルを取得

from keras.models import load_model

model = load_model('ex1-result/model2.h5')
model.load_weights('ex1-result/finetuning2.h5')


In [17]:
_, validation_generator = image_generator()
loss, acc = model.evaluate_generator(validation_generator, steps=nb_validation_samples/batch_size)
print('accuracy:', acc)

Found 2000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
accuracy: 0.961666665872


今回の試行では最高スコアは0.9616でしたが、他の試行では0.971666665872のスコアを得ることができました。

0,32,0.96  
1,64,0.866666665872  
2,16,0.956666666667  
3,32,0.954999999205  
4,32,**0.971666665872**  
5,16,0.963333333333  
6,4,0.916666666667  
7,64,0.936666665872  
8,16,0.955  
9,32,0.948333334128  
10,16,0.961666666667  
11,4,0.5  
12,4,0.955  
13,16,0.963333333333  
14,32,**0.971666666667**  
15,16,0.956666666667
